In [22]:
import warnings
warnings.filterwarnings('ignore')
import xml.etree.ElementTree as ET
import sys
import os
import pandas as pd
import json
import xmltodict
import nltk, re

In [3]:
DATA_DIR = "MedQuAD/1_CancerGov_QA"

DATA_FILES = sorted(os.listdir(DATA_DIR))

In [4]:
elemList = []
for each_file in DATA_FILES:
    tree = ET.parse(DATA_DIR+'/'+each_file)
    for elem in tree.iter():
        elemList.append(elem.tag)

elemList = list(set(elemList))

print(elemList)

['Answer', 'FocusAnnotations', 'UMLS', 'CUI', 'Question', 'CUIs', 'Document', 'QAPairs', 'QAPair', 'SemanticGroup', 'SemanticTypes', 'SemanticType', 'Focus']


In [5]:
data_column_names = ["Focus",'qtype', "Question",  "Answer"]

QA_DATAFRAME = pd.DataFrame(columns = data_column_names)
QA_SIZE = 0
for each_file in DATA_FILES:
    tree = ET.parse(DATA_DIR+'/'+each_file)
    root = tree.getroot()
    SemanticTypes = []
    SemanticGroups = []
    QTypes = []
    Questions = []
    Answers = []
    doc_focus = []

    for each_passage in root.iter('Document'):
        doc_focus.append(each_passage.find('Focus').text)
        
        for each_Question in each_passage.iter('Question'):
            QTypes.append(each_Question.attrib['qtype'])
            Questions.append(each_Question.text)
            
        for each_Answer in each_passage.iter('Answer'):
            temp_answer = each_Answer.text.replace('\n', ' ').replace('\t', '')
            Answers.append(temp_answer)
    
    doc_df = pd.DataFrame(columns = data_column_names)
    if (len(Questions) == len(Answers)):
        for index in range(len(Questions)):
            if (len(SemanticTypes) == 1):
                SemanticType = SemanticTypes[0]
            elif (len(SemanticTypes) == 2):
                SemanticType = SemanticTypes[0] + ',' + SemanticTypes[1]
            temp_df = pd.DataFrame([[doc_focus[0], QTypes[index], Questions[index], Answers[index]]], columns=data_column_names)
            doc_df = doc_df.append(temp_df, ignore_index=True)
        QA_SIZE += len(Questions)
    QA_DATAFRAME = QA_DATAFRAME.append(doc_df, ignore_index=True)

In [6]:
QA_DATAFRAME

,Focus,qtype,Question,Answer
0,Adult Acute Lymphoblastic Leukemia,information,What is (are) Adult Acute Lymphoblastic Leukem...,Key Points - Adult acute l...
1,Adult Acute Lymphoblastic Leukemia,symptoms,What are the symptoms of Adult Acute Lymphobla...,"Signs and symptoms of adult ALL include fever,..."
2,Adult Acute Lymphoblastic Leukemia,exams and tests,How to diagnose Adult Acute Lymphoblastic Leuk...,Tests that examine the blood and bone marrow a...
3,Adult Acute Lymphoblastic Leukemia,outlook,What is the outlook for Adult Acute Lymphoblas...,Certain factors affect prognosis (chance of re...
4,Adult Acute Lymphoblastic Leukemia,susceptibility,Who is at risk for Adult Acute Lymphoblastic L...,Previous chemotherapy and exposure to radiatio...
...,...,...,...,...
724,Vulvar Cancer,exams and tests,How to diagnose Vulvar Cancer ?,Tests that examine the vulva are used to detec...
725,Vulvar Cancer,outlook,What is the outlook for Vulvar Cancer ?,Certain factors affect prognosis (chance of re...
726,Vulvar Cancer,stages,What are the stages of Vulvar Cancer ?,Key Points - After vulvar ...
727,Vulvar Cancer,research,what research (or clinical trials) is being do...,New types of treatment are being tested in cli...


In [9]:
QA_DATAFRAME.to_csv("xml_table.tsv", sep="\t", index=False)

In [43]:
QA_DATAFRAME = QA_DATAFRAME[["qtype", "Question",  "Answer"]]
LOL = QA_DATAFRAME.values.tolist()

fh = open("QA_RAW_text.txt", "w")
ID = 1
for L in LOL:
    #print(L)
    qtype, Question,  Answer = L
    print(">", ID, file=fh)
    print("$" + qtype, file=fh)
    print("$" + Question, file=fh)
    
    Answer = Answer.replace("Key Points - ", "").replace("-", "")
    Answer = re.sub(' +', ' ', Answer) # get rid of extra speces
    print("$" + Answer, file=fh)
    Answers = ["#" + i for i in nltk.sent_tokenize(Answer)]
    for a in Answers:print(a, file=fh)
        
    ID += 1
    #break
fh.close()